In [ ]:
!pip install pinecone-client datasets openai llama_index llama-index-vector-stores-pinecone

In [ ]:
import os

import time
os.environ['PINECONE_API_KEY']='<YOUR_API_KEY>'
os.environ['OPENAI_API_KEY']='<YOUR_API_KEY>'
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
# 기반 데이터셋 로드
from datasets import load_dataset
dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:100]')

In [ ]:
# 라마인덱스 네이티브 형태의 다큐먼트로 파싱하기 위해 데이터프레임으로 정의
data = dataset.to_pandas()[['id', 'text', 'title']]
data.drop_duplicates(subset='text', keep='first', inplace=True)
data.head()

In [ ]:
#라마인덱스 다큐먼트 오브젝트로서 텍스트 데이터를 메타와 함께 정의
from llama_index.core import Document

docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        doc_id=row['id'],
        extra_info={'title': row['title']}
    ))



In [ ]:
# Document 확인
docs[0]

In [ ]:
# 노드로 청킹
from llama_index.core.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(docs)
nodes[0]

In [ ]:
# 파인콘 연결
from pinecone import Pinecone, PodSpec

api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=api_key)

In [ ]:
from time import sleep

index_name = 'quickstart'
dimension = 1536
metric = 'dotproduct'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)
index = pc.Index(index_name)
sleep(1)
index_stats = index.describe_index_stats()
print(index_stats)

In [ ]:
# 라마인덱스 벡터스토어로 정의
from llama_index.vector_stores.pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(pinecone_index=index)

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding

# 벡터스토어 인덱스에 들어가는 스토리지 컴포넌트 정의
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
# 서비스 컨텍스트에 해당 인덱스를 활용한 연결 컴포넌트 정의(임베딩 모델 +@)
embed_model = OpenAIEmbedding(model='text-embedding-ada-002', embed_batch_size=100)
service_context = ServiceContext.from_defaults(embed_model=embed_model)

index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context,
    service_context=service_context
)

In [ ]:
query_engine = index.as_query_engine()
res = query_engine.query("지미 카터가 누구야?")
print(res)